# Agents - Basic
An Agent is like a smart decision-maker.
Instead of just following a fixed script (step 1 → step 2 → step 3), an agent can decide what to do next based on the situation.

It works like this:

- You give the agent a goal (e.g., "Find me the weather in New York").
- The agent thinks: "Do I already know this? Or should I call a tool?"
- It can use tools (like a calculator, database, or API) to get the answer.
- It then returns the final result to you.

In [3]:
! pip install langchain-google-genai

In [4]:
# Importing Necessary Libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.agents import create_react_agent, AgentExecutor, tool
from langchain import hub
import datetime
from google.colab import userdata


In [5]:
# Getting the API KEY
API_KEY = userdata.get('GOOGLE_API_KEY')


In [6]:
# Choosing the model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    google_api_key = API_KEY
)


In [9]:
# Providing a query
query = "What is the current time?"

# Converting template to prompt template
prompt_template = PromptTemplate.from_template("{input}")

# Making Chain
chain = prompt_template | llm | StrOutputParser()

# Generating Response
response = chain.invoke({
    "input": query
})

# Printing
print(response)

I do not have access to the current time. To find out the current time, please check your phone, computer, or another time-telling device.


In [10]:
# Providing a query
query = "What is the current time?"

# hub.pull("hwchase17/react") - This loads a special prompt style called ReAct.
prompt_template = hub.pull("hwchase17/react")

# The agent has no extra tools to use (like calculator, search, etc.)
tools = []

# Creating the agent
agent = create_react_agent(llm, tools, prompt_template)

# Setting up an executor to run the agent
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

# Generating response
response = agent_executor.invoke({
    "input": query
})

# Printing
print(response)



> Entering new AgentExecutor chain...
I need to find the current time. I don't have a specific tool for that, so I'll use the browser tool to search for it.
Action: browser
Action Input: current timebrowser is not a valid tool, try one of [].I need to find the current time. I don't have a specific tool for that, so I'll try searching the web.
Action: search
Action Input: current timesearch is not a valid tool, try one of [].I apologize, I do not have the capability to access the current time. I am unable to use external tools or access the internet.
Final Answer: I am sorry, I cannot provide the current time.


> Finished chain.
{'input': 'What is the current time?', 'output': 'I am sorry, I cannot provide the current time.'}


In [17]:
# Providing tools to get proper response
@tool

# Defining a function for tools
def system_time(format: str =  "%Y-%m-%d %H:%M:%S" ):
   """ Returns the current date and time in the specified format """

   # Getting current time
   current_time = datetime.datetime.now()

   # Converting to desired format
   formatted_time = current_time.strftime(format)
   return formatted_time

# Providing a query
query = "What is the current time?"

# hub.pull("hwchase17/react") - This loads a special prompt style called ReAct.
prompt_template = hub.pull("hwchase17/react")

# The agent has no extra tools to use (like calculator, search, etc.)
tools = [system_time]

# Creating the agent
agent = create_react_agent(llm, tools, prompt_template)

# Setting up an executor to run the agent
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

# Generating response
response = agent_executor.invoke({
    "input": query
})

# Printing
print(response)



> Entering new AgentExecutor chain...
I need to get the current time to answer the question.
Action: system_time
Action Input: %Y-%m-%d %H:%M:%S2025-09-22 15:54:08I now know the current time.
Final Answer: The current time is 2025-09-22 15:54:08.

> Finished chain.
{'input': 'What is the current time?', 'output': 'The current time is 2025-09-22 15:54:08.'}


In [18]:
# Doing the same with a different query
# Providing tools to get proper response
@tool

# Defining a function for tools
def system_time(format: str =  "%Y-%m-%d %H:%M:%S" ):
   """ Returns the current date and time in the specified format """

   # Getting current time
   current_time = datetime.datetime.now()

   # Converting to desired format
   formatted_time = current_time.strftime(format)
   return formatted_time

# Providing a query
query = "What is the current time in New-York? (You are in pakistan). Just show the current time and not the date."

# hub.pull("hwchase17/react") - This loads a special prompt style called ReAct.
prompt_template = hub.pull("hwchase17/react")

# The agent has no extra tools to use (like calculator, search, etc.)
tools = [system_time]

# Creating the agent
agent = create_react_agent(llm, tools, prompt_template)

# Setting up an executor to run the agent
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

# Generating response
response = agent_executor.invoke({
    "input": query
})

# Printing
print(response)



> Entering new AgentExecutor chain...
I need to find the current time, but I only have access to a tool that provides the current time in my location (Pakistan). Since I need the time in New York, I need to adjust for the time difference. I'll first get the current time in Pakistan, then calculate the time difference between Pakistan and New York, and finally subtract that difference from the Pakistan time to get the New York time.

Action: system_time
Action Input: %H:%M:%S15:54:20Okay, the current time in Pakistan is 15:54:20. New York is 9 hours behind Pakistan. So, I need to subtract 9 hours from the current time in Pakistan. 15 - 9 = 6. Therefore, the current time in New York is 06:54:20.
Final Answer: 06:54:20


> Finished chain.
{'input': 'What is the current time in New-York? (You are in pakistan). Just show the current time and not the date.', 'output': '06:54:20'}
